In [ ]:
import pandas as pd, boto3, json
from io import BytesIO
from datetime import datetime

In [14]:
with open("/home/jh97/MyWorks/Documents/.aws_cdt.json", "r") as file:
    content = json.load(file)
    key_id = content["access_key"]
    secret_key = content["secrect_access_key"]

client = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
)

# panigator = client.get_paginator("list_objects_v2")
# result = panigator.paginate(Bucket="crawling-to-datalake", Prefix="scraped/")

# for i in result.build_full_result()["Contents"]:
#     if datetime.fromisoformat(
#         i["Key"].split("_")[-1].split(".")[0]
#     ) < datetime.now() and i["Key"].split("_")[0].endswith("tgdd"):
#         print(i["Key"])
#         break

resp = client.get_object(
    Bucket="crawling-to-datalake", Key="scraped/tgdd_products_2025-08-26.csv"
)

df = pd.read_csv(BytesIO(resp["Body"].read()))
df

,id,name,onsale_price,brand,category,os,cpu,gpu,rating,reviews_count,is_new,url,retailer,created_at,updated_at
0,207956,Điện thoại Nokia 220 4G,980000,Nokia,Smartphone,S30+,NaN,NaN,4.9,61302.0,True,https://www.thegioididong.com/dtdd/nokia-220-4g,TGDD,06/2024,2025-08-26 15:07:46
1,281570,Điện thoại iPhone 15 128GB,15290000,iPhone (Apple),Smartphone,iOS 17,NaN,NaN,4.9,91248.0,True,https://www.thegioididong.com/dtdd/iphone-15,TGDD,09/2023,2025-08-26 15:07:46
2,333360,Điện thoại Samsung Galaxy S25 Plus 5G 12GB/512GB,24440000,Samsung,Smartphone,Android 15,NaN,NaN,5.0,657.0,True,https://www.thegioididong.com/dtdd/samsung-gal...,TGDD,01/2025,2025-08-26 15:07:46
3,334690,Laptop GIGABYTE Gaming G5 MF5 RC555 - 9RC55MF5...,22790000,GIGABYTE,Laptop,Windows 11 Home SL,Intel Core i7 Raptor Lake - 13620H,"Card rời - NVIDIA GeForce RTX 4050, 6 GB",5.0,126.0,True,https://www.thegioididong.com/laptop/gigabyte-...,TGDD,2023,2025-08-26 15:07:46
4,335638,Điện thoại Xiaomi Redmi 13x 8GB/128GB,4210000,Xiaomi,Smartphone,Xiaomi HyperOS (Android 14),NaN,NaN,4.9,33117.0,True,https://www.thegioididong.com/dtdd/xiaomi-redm...,TGDD,03/2025,2025-08-26 15:07:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,336409,Điện thoại Masstel S9 6GB/256GB,2940000,Masstel,Smartphone,Android 14,NaN,NaN,4.9,1733.0,True,https://www.thegioididong.com/dtdd/masstel-s9,TGDD,04/2025,2025-08-26 15:07:46
1119,319214,Điện thoại vivo V30 5G 12GB/512GB,13740000,vivo,Smartphone,Android 14,NaN,NaN,4.9,5108.0,True,https://www.thegioididong.com/dtdd/vivo-v30,TGDD,05/2024,2025-08-26 15:07:46
1120,333352,Điện thoại Samsung Galaxy S25 Ultra 5G 12GB/1TB,34680000,Samsung,Smartphone,Android 15,NaN,NaN,5.0,901.0,True,https://www.thegioididong.com/dtdd/samsung-gal...,TGDD,01/2025,2025-08-26 15:07:46
1121,329151,Điện thoại iPhone 16 Pro Max 1TB,42690000,iPhone (Apple),Smartphone,iOS 18,NaN,NaN,5.0,8083.0,True,https://www.thegioididong.com/dtdd/iphone-16-p...,TGDD,09/2024,2025-08-26 15:07:46


In [114]:
df = pd.read_csv("../data/scraped/tgdd_products_2025-08-26.csv")
df
df.dtypes

id                 int64
name              object
onsale_price       int64
brand             object
category          object
os                object
cpu               object
gpu               object
rating           float64
reviews_count    float64
is_new              bool
url               object
retailer          object
created_at        object
updated_at        object
dtype: object

In [312]:
# df["name"].str.replace(
#     r"^Điện thoại\s+|^Laptop\s+|\s+-.*|\s+\d+(GB|TB).*",
#     "",
#     regex=True,
#     case=False,
# ).sort_values().unique()

df["brand"].where(
    ~df["brand"].str.contains(r"iphone|macbook", regex=True, case=False), other="Apple"
)

df["os"].str.replace(r"\s+\+.*|\s+\(.*", "", regex=True, case=False)

df["cpu"].str.replace(r"-\sHãng.*|chuỗi N\s+", "", regex=True, case=False)


df["gpu"][
    ~df["gpu"].astype(str).str.contains(r"rời|NVIDIA", regex=True, case=False)
].unique()

df["gpu"].map(
    lambda x: (
        (
            x + ", Discrete"
            if ("NVIDIA" in str(x) or "rời" in str(x))
            else x + ", Integrated"
        )
        if isinstance(x, str)
        else None
    )
).str.replace(r".*\s+-\s+|\s+eligible|\s+\(.*\)", "", regex=True, case=False).unique()

array([None, 'NVIDIA GeForce RTX 4050, 6 GB, Discrete',
       'AMD Radeon 840M, Integrated',
       'NVIDIA GeForce RTX 3050, 4 GB, Discrete',
       'Intel Arc 140T, Integrated',
       'NVIDIA GeForce RTX 5050, 8 GB, Discrete',
       'Intel Iris Xe Graphics, Integrated', 'Intel Graphics, Integrated',
       'Intel UHD Graphics, Integrated', 'Intel Arc Graphics, Integrated',
       'AMD Radeon 860M, Integrated',
       'NVIDIA GeForce RTX 4060, 8 GB, Discrete',
       'AMD Radeon Graphics, Integrated', '10 nhân GPU, Integrated',
       '8 nhân GPU, Integrated',
       'NVIDIA GeForce RTX 3050, 6 GB, Discrete',
       'NVIDIA GeForce RTX 2050, 4 GB, Discrete',
       'NVIDIA GeForce RTX 4070, 8 GB, Discrete',
       'NVIDIA GeForce RTX 5060, 8 GB, Discrete',
       'AMD Radeon 610M Graphics, Integrated', '20 nhân GPU, Integrated',
       '16 nhân GPU, Integrated', 'AMD Radeon RX 6550M, 4 GB, Discrete',
       '7 nhân GPU, Integrated', '32 nhân GPU, Integrated',
       'NVIDIA GeForce